In [1]:
import glob
import os
import pandas as pd
import shutil
import re

In [55]:
def rename_and_copy_file(mvt_file, destination_folder):
    # Create a copy of the original file
    copied_file = os.path.join(destination_folder, os.path.basename(mvt_file))
    shutil.copy2(mvt_file, copied_file)

    # Rename the copied file to .csv
    csv_file = os.path.splitext(copied_file)[0] + '.csv'
    os.rename(copied_file, csv_file)

    # Return the path to the renamed .csv file
    return csv_file

In [56]:
# Specify the directory where the files are located
directory = "C:\\Users\\AS81970\\Desktop\\Q2 2023\\FAO Needs"

# Specify the pattern to match the files (including the common part of the filename)
file_pattern = "014_adc_pos_daily_*.mvt"

# Find all files matching the pattern
files = glob.glob(os.path.join(directory, file_pattern))

# Sort the files by modification time (most recent file comes first)
files.sort(key=os.path.getmtime, reverse=False)

# Get the path of the latest file
latest_file = files[0]

In [57]:
# Rename and copy the latest file to the destination folder
destination_folder = "C:\\Users\\AS81970\\Desktop\\Q2 2023\\FAO Needs\\Converted"
copied_csv_file = rename_and_copy_file(latest_file, destination_folder)

In [58]:
print(copied_csv_file)

C:\Users\AS81970\Desktop\Q2 2023\FAO Needs\Converted\014_adc_pos_daily_20230717.csv


In [59]:
# Read the data from the CSV file, skipping the rows based on a condition
df = pd.read_csv(copied_csv_file, sep=',', header=None, skiprows=lambda x: str(x) == '0' or (isinstance(x, str) and x.startswith('@')))

In [60]:
df.head()

,0,1,2,3,4,5,6,7,8,9
0,1400335,20230716083800,NaN,NaN,21635000000,W,1.05,7.34,NaN,NaN
1,1400335,20230716083800,NaN,NaN,22594100000,W,0.56,7.83,NaN,NaN
2,1400335,20230716083800,NaN,NaN,23583540000,W,1.07,10.15,NaN,NaN
3,1400335,20230716084900,NaN,NaN,23710010000,W,0.91,5.45,NaN,NaN
4,1400335,20230716091000,NaN,NaN,20564890000,W,0.73,7.66,NaN,NaN


In [61]:
# Set the column names
df.columns = ['Store', 'Timestamp', 'Column1', 'Column2', 'Item', 'Column3', 'Column4', 'Column5', 'Column6','Column7']
df['Item'] = df['Item'].astype(str)

In [62]:
df.head()

,Store,Timestamp,Column1,Column2,Item,Column3,Column4,Column5,Column6,Column7
0,1400335,20230716083800,NaN,NaN,21635000000,W,1.05,7.34,NaN,NaN
1,1400335,20230716083800,NaN,NaN,22594100000,W,0.56,7.83,NaN,NaN
2,1400335,20230716083800,NaN,NaN,23583540000,W,1.07,10.15,NaN,NaN
3,1400335,20230716084900,NaN,NaN,23710010000,W,0.91,5.45,NaN,NaN
4,1400335,20230716091000,NaN,NaN,20564890000,W,0.73,7.66,NaN,NaN


In [63]:
# Define the list of items to filter
item_list = ["22808800000" ,'26859300000','0026859300000',"23858800000" ,"23889500000" ,"26807200000" ,"26807400000" ,"26807600000" ,"26807800000" ,"26859000000" ,"26859300000" ,"26865400000" ,"26865500000" ,"26871000000" ,"26889700000" ,"26908100000" ,"26908900000" ,"26909000000" ,"26909100000" ,"26910100000" ,"26911000000" ,"26911400000" ,"26912400000" ,"26913700000" ,"26916800000" ,"26917300000" ,"26918200000" ,"26918300000" ,"26919000000" ,"26919200000" ,"26924200000" ,"26924300000" ,"28807400000" ,"28807600000" ,"28889500000"]
item_list = [str(item) for item in item_list] 

In [64]:
# Filter rows based on the store code
store_code = '1400444'
store_code = store_code.strip()  # Remove leading/trailing spaces
df['Store'] = df['Store'].astype(str).str.strip()

In [65]:
filtered_df = df[(df['Store'] == store_code) & (df['Item'].isin(item_list))]

In [66]:
filtered_df.head()

,Store,Timestamp,Column1,Column2,Item,Column3,Column4,Column5,Column6,Column7
24949,1400444,20230716090100,NaN,NaN,26917300000,W,1.04,12.47,NaN,NaN
24950,1400444,20230716090100,NaN,NaN,26917300000,W,0.90,10.79,NaN,NaN
24951,1400444,20230716090100,NaN,NaN,26917300000,W,1.04,12.47,NaN,NaN
24952,1400444,20230716090100,NaN,NaN,26917300000,W,0.90,10.79,NaN,NaN
24989,1400444,20230716091800,NaN,NaN,26859300000,W,0.96,11.51,NaN,NaN


In [67]:
sorted_df = filtered_df.sort_values(by=['Item'])

In [68]:
# Extract the timestamp from the file name using regular expressions
timestamp = re.search(r'\d{8}', copied_csv_file).group()

In [69]:
print(copied_csv_file)

C:\Users\AS81970\Desktop\Q2 2023\FAO Needs\Converted\014_adc_pos_daily_20230717.csv


In [70]:
print(timestamp)

20230717


In [71]:
# Define the Excel file name with the timestamp
excel_file_name = f'SF_1400444_{timestamp}.xlsx'

# Define the output directory path
output_directory = r'C:\Users\AS81970\Desktop\Q2 2023\FAO Needs\Reports\\'

# Combine the output directory path and the excel file name
full_file_path = os.path.join(output_directory, excel_file_name)

# Create a Pandas Excel writer
writer = pd.ExcelWriter(full_file_path, engine='xlsxwriter')

# Save the sorted DataFrame to the Excel file with custom number format
sorted_df.to_excel(writer, index=False, sheet_name='Sheet1')

# Get the workbook and worksheet objects
workbook = writer.book
worksheet = writer.sheets['Sheet1']

# Set the number format for the timestamp column
timestamp_format = workbook.add_format({'num_format': '0'})
worksheet.set_column('B:B', None, timestamp_format)

# Close the Pandas Excel writer
writer.save()



C:\Users\AS81970\AppData\Local\Temp\ipykernel_23172\4187051172.py:25: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
